#Study FT1 files


In [1]:
import os, glob
infolder = os.path.expandvars('$FERMI/data/P8_P301/Source/')
ft1s = sorted(glob.glob(infolder+'*Source*.fits'))
print 'found %d FT1 files' % len(ft1s)

found 73 FT1 files


In [2]:
import pyfits
ft1f = pyfits.open(ft1s[0])
print ft1f.info()
print ft1f[1].columns

Filename: /afs/slac/g/glast/groups/catalog//pointlike/fermi/data/P8_P301/Source/P301_Source_001_zmax100.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      32   ()              
1    EVENTS      BinTableHDU    206   2235172R x 23C   [E, E, E, E, E, E, E, E, E, D, J, J, I, 3I, 32X, 32X, I, D, E, E, E, E, E]   
2    GTI         BinTableHDU     48   417R x 2C    [D, D]   
None
ColDefs(
    name = 'ENERGY'; format = 'E'; unit = 'MeV'
    name = 'RA'; format = 'E'; unit = 'deg'
    name = 'DEC'; format = 'E'; unit = 'deg'
    name = 'L'; format = 'E'; unit = 'deg'
    name = 'B'; format = 'E'; unit = 'deg'
    name = 'THETA'; format = 'E'; unit = 'deg'
    name = 'PHI'; format = 'E'; unit = 'deg'
    name = 'ZENITH_ANGLE'; format = 'E'; unit = 'deg'
    name = 'EARTH_AZIMUTH_ANGLE'; format = 'E'; unit = 'deg'
    name = 'TIME'; format = 'D'; unit = 's'
    name = 'EVENT_ID'; format = 'J'
    name = 'RUN_ID'; format = 'J'
    name = 'RECON_VERSION'

###Extract and study bits in  EVENT_TYPE

In [3]:
#convert low 6 bits to integer
powers = np.array([2**(5-n) for n in range(6)])
data=ft1f[1].data
et_bits=data['EVENT_TYPE']
et = np.array([sum(powers * np.array(x[-6:],int)) for x in et_bits])
#front is the lowest order bit
front = (et&1)==0; sum(front)
psf_index = et/4; 
psf_totals = [ sum( psf_index==x ) for x in [1,2,4,8] ]
psf_totals, sum(psf_totals), len(et)

###Check correlation between PSF0,1,2,3 and FRONT

In [5]:
psf_and_front = [ sum( (psf_index==x) & front ) for x in [1,2,4,8] ]
print psf_and_front
(np.array(psf_and_front)/np.array(psf_totals,float)).round(2)

[467606, 355846, 161305, 3946]


###Create a Data object

In [ ]:
import pointlike
pd=    pointlike.Data(ft1s[0], -1, 0, 0, -1, '')
pd

In [13]:
psf= np.array((2.0, 4.6, 5.8, 8.1))
fb= np.array((3.5, 6.6))
cur =  np.degrees(np.array([0.0116,0.0192] ))
(fb/cur, (np.radians(psf/5))[::-1].round(4))


(array([ 5.26607964,  5.9995693 ]),
 array([ 0.0283,  0.0202,  0.0161,  0.007 ]))